# **Análisis exploratorio de datos**

El Análisis Exploratorio de Datos (EDA) tiene como objetivo principal comprender lo que los datos revelan y descubrir patrones o ideas dentro del conjunto de datos antes de proceder al modelado estadístico formal o a la formulación y prueba de hipótesis.

Voy a dividir el análisis en los siguientes apartados:
Analisis del problema.

1. Análisis del problema.
2. Análisis univariable.
3. Análisis multivariable.
4. Limpieza de datos.
5. Comprobación de supuestos.







## Librerías necesarias.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
import warnings
from IPython.display import Image
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

In [ ]:
house_df = pd.read_csv("Data/houseprices_train.csv")

In [ ]:
house_df.head()

# 1.Análisis del problema


Para asimilar verdaderamente la información contenida en el conjunto de datos, procederemos a examinar el significado y la importancia de cada característica en relación con el escenario planteado. Buscaremos responder preguntas fundamentales, tales como:

*   La influencia de la característica en el costo final de una propiedad.
*  Duplicidad de información con otras características.

La finalidad del proyecto es la estimación del valor de las propiedades inmobiliarias, por lo tanto, el factor principal a predecir es el Valor de la Vivienda.

In [ ]:
#El conjunto de datos inicial consta de 81 variables, pero para los fines de este análisis, nos enfocaremos únicamente en un subconjunto de 25 columnas seleccionadas.
variables_eliminar = [
'LandSlope', 'LotFrontage', 'LotShape', 'LandContour', 'Condition1', 'Condition2', 'RoofStyle', 'RoofMatl',
'Exterior2nd', 'ExterQual', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageType',
'GarageYrBlt', 'GarageFinish', 'PavedDrive', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
'ScreenPorch', 'Fence', 'MiscFeature', 'MiscVal', 'Street', 'Alley', 'LotConfig', 'Exterior1st', 'MasVnrType',
'ExterCond', 'HeatingQC', 'CentralAir', 'FireplaceQu', 'GarageCond', 'HalfBath', 'Fireplaces', 'GarageCars',
'WoodDeckSF', 'YearBuilt', 'OverallQual', 'Heating', 'MasVnrArea', 'Electrical', 'KitchenQual', 'GarageQual',
'MSSubClass']
house_df.drop(columns = variables_eliminar, axis = 1, inplace = True)

In [ ]:
#Renombramiento de columnas
dict_renombre = {
'MSZoning':'Zonificacion', 'LotArea':'TamanioLote', 'Utilities':'ServiciosBasicos', 'Neighborhood':'Vecindario',
'BldgType':'TipoVivienda', 'HouseStyle':'EstiloVivienda', 'OverallCond':'EstadoCasa',
'YearRemodAdd':'FechaRemodelacion', 'Foundation':'TipoCimentacion', '1stFlrSF':'AreaPiso1', '2ndFlrSF':'AreaPiso2',
'GrLivArea':'SuperficieHabitable', 'FullBath':'Banios', 'BedroomAbvGr':'Dormitorios', 'KitchenAbvGr':'Cocinas',
'TotRmsAbvGrd':'Habitaciones', 'GarageArea':'AreaGaraje', 'PoolArea': 'AreaPiscina', 'PoolQC':'CalidadPiscina','MoSold':'MesVenta',
'YrSold':'AnioVenta', 'SaleType':'TipoVenta', 'SaleCondition':'CondicionVenta', 'SalePrice':'PrecioVenta'
}
house_df.rename(columns = dict_renombre, inplace = True)

In [ ]:
house_df.head()

Separamos las variables independientes en variables cuantitativas y cualitativas

In [ ]:
quantitative = house_df.select_dtypes(exclude = 'object').columns.to_list()
#PrecioVenta es la variable dependiente, por ello lo eliminamos de quantitative.
quantitative.remove('PrecioVenta')
#Id solo es un identificador del registro, por ello lo eliminamos quantitative.
quantitative.remove('Id')
qualitative = house_df.select_dtypes(include = 'object').columns.to_list()

# 2.Análisis univariable

In [ ]:
# Resumen de la estadística descriptiva:
house_df['PrecioVenta'].describe()

In [ ]:
# Histograma de la distribución de los precios de las viviendas.
sns.distplot(house_df['PrecioVenta'], fit = norm)
plt.plot()

Al observar el gráfico, se pueden identificar inmediatamente las siguientes características:



*   Una discrepancia en comparación con la curva de distribución normal
*   Una cola extendida hacia la derecha (sesgo positivo).
*   La presencia de valores concentrados y prominentes (o modas).



In [ ]:
# Asimetría y curtosis:
print("Skewness: %f" % house_df['PrecioVenta'].skew())
print("Kurtosis: %f" % house_df['PrecioVenta'].kurt())

La Curtosis es mayor que cero,  entonces la curva es leptocúrtica, por lo que hay mayor probabilidad de encontrar valores atípicos.

In [ ]:
# Diagrama de dispersión
def scatterplot_func(x, y, **kwargs):
    sns.scatterplot(x=x, y=y, alpha = 0.5)
    x=plt.xticks(rotation=90)

# Le aplicamos unpivot al dataframe por cada variable cuantitativa
f = pd.melt(frame = house_df, id_vars=['PrecioVenta'], value_vars=quantitative)
# Creamos las parcelas donde colocaremos nuestros gráficos
g = sns.FacetGrid(data = f, col="variable",  col_wrap=2, sharex=False, sharey=False)
# Poblamos las parcelas con los gráficos
g = g.map(scatterplot_func, "value", "PrecioVenta")

Conclusión:

'AreaPiso1' y 'SuperficieHabitable' mantienen una relación lineal positiva con 'PrecioVenta', aumentando en el mismo sentido.

### Variables cualitativas

Con variables cualitativas podemos implementar dos métodos:  
1. Verificar la distribución de PrecioVenta con respecto a los valores de las variables y enumerarlos.
2. Crear una variable ficticia para cada categoría posible.

In [ ]:
def variable_categorica(df, qualitative):
  for c in qualitative:
    #convertimos a categoría
    df[c] = df[c].astype('category')
    if(df[c].isnull().any()):
      #Aniadimos la categoría missing en caso existan valores nan
      df[c] = df[c].cat.add_categories(['MISSING'])
      df[c] = df[c].fillna('MISSING')
variable_categorica(house_df, qualitative)

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

# Le aplicamos unpivot al dataframe por cada variable cualitativa
f = pd.melt(house_df, id_vars=['PrecioVenta'], value_vars=qualitative)
# Creamos las parcelas donde colocaremos nuestros gráficos
g = sns.FacetGrid(f, col="variable", col_wrap=2, sharex=False, sharey=False, aspect=1.5)
# Poblamos las parcelas con los gráficos
g = g.map(boxplot, "value", "PrecioVenta")

Conclusiones Reafirmadas

* La ubicación/zona ejerce una influencia significativa sobre la cotización de las propiedades inmobiliarias.


* La variable que indica una condición de venta parcial (incompleta) está asociada al valor más elevado.

* La inclusión de una piscina en la vivienda se correlaciona con un incremento sustancial en su valor de mercado.

* Se observan disparidades en la dispersión (o varianza) de los precios entre las distintas agrupaciones de categorías analizadas.

### Test ANOVA

¿El Vecindario tiene alguna repercusión en el precio?

* Hipótesis nula (H0): No hay diferencias significativas en los precios de venta entre los diferentes vecindarios
* Hipótesis alternativa (H1): Existen diferencias significativas en los precios de venta entre al menos dos vecindarios

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('PrecioVenta ~ Vecindario', data=house_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

p-value < 0.05, por lo tanto se rechaza la hipótesis nula. Significa que hay evidencia suficiente para decir que al menos dos vecindarios tienen precios de venta diferentes.

### Encoding
Una forma de encodear es creando variables dummy(binarias).

In [ ]:
house_df_dummies = pd.get_dummies(data = house_df, prefix_sep='_', columns=qualitative)

In [ ]:
house_df["Zonificacion"].unique()

In [ ]:
palabra_buscar = "Zonificacion"
columnas_contiene_palabra = [col for col in house_df_dummies.columns.to_list() if palabra_buscar in col]
house_df_dummies[columnas_contiene_palabra].sample(5)

* Identificación del Patrón: Las categorías analizadas muestran una estructura inherentemente ordenada. Un caso claro es la característica "piscina", donde una vivienda con piscina establece una calidad o jerarquía superior respecto a una sin ella.

* Decisión de Modelado: Para capturar este orden y representarlo numéricamente, se implementará una técnica de codificación: transformaremos estas variables ordinales (o con orden implícito) asignando a cada categoría el valor de la media del precio de venta observada en dicha categoría.

In [ ]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    #Guardamos los valores únicos de cada variable cualitativa
    ordering['ValoresUnicos'] = frame[feature].unique()
    #Los índices de nuestro df serán estas categorías
    ordering.index = ordering.ValoresUnicos
    #Hallamos la media del precio de venta por cada valor único de la variable categórica
    ordering['PrecioVentaMedia'] = frame[[feature, 'PrecioVenta']].groupby(feature).mean()['PrecioVenta']
    #Ordenamos nuestro df de acuerdo a la media del precio de venta
    ordering = ordering.sort_values('PrecioVentaMedia')
    #Asignamos un valor de orden a cada valor único
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()

    #Creamos nuestros campos encodeados en nuestro dataframe
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o

#Guardamos los nombres de columnas encodeadas en una lista
qual_encoded = []
for q in qualitative:
    encode(house_df, q)
    qual_encoded.append(q+'_E')
#Mostramos los campos encodeados
print(qual_encoded)

In [ ]:
house_df[["TipoCimentacion", "TipoCimentacion_E"]].sample(5)

# 3.Análisis multivariable

In [ ]:
# Matriz de correlación:
corrmat = house_df[qual_encoded + quantitative + ["PrecioVenta"]].corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
#El mismo diagrama pero por partes
sns.set(font_scale = 1.0)
plt.figure(1)
corr = house_df[quantitative+['PrecioVenta']].corr()
sns.heatmap(corr)

plt.figure(2)
corr = house_df[qual_encoded+['PrecioVenta']].corr()
sns.heatmap(corr)

plt.figure(3)
corr = pd.DataFrame(np.zeros([len(quantitative)+1, len(qual_encoded)+1]),
                    index=quantitative+['PrecioVenta'], columns=qual_encoded+['PrecioVenta'])
for q1 in quantitative+['PrecioVenta']:
    for q2 in qual_encoded+['PrecioVenta']:
        corr.loc[q1, q2] = house_df[q1].corr(house_df[q2])
sns.heatmap(corr)

## Matriz de correlación con la variable PrecioVenta

In [ ]:
corr = house_df[quantitative + qual_encoded + ["PrecioVenta"]].corr()[['PrecioVenta']]
corr.sort_values(by = 'PrecioVenta',ascending = False).style.background_gradient()

**Conclusión**  
- Hay muchas correlaciones fuertes entre las variables.
- El vecindario se correlaciona con muchas otras variables y esto confirma la idea de que las casas en la misma región comparten las mismas características.
- El tipo de vivienda se correlaciona negativamente con el número de cocinas.
- 'Banios' también está correlacionada con 'PrecioVenta', 'Habitaciones' y 'SuperficieHabitable'. Tenemos **multicolinealidad**
- 'Habitaciones' y 'SuperficieHabitable', otro caso de **multicolinealidad**

### Multicolinealidad

* La fuerte relación entre las variables que usamos para predecir (variables predictoras) es un problema.

* Esta correlación excesiva tiene el potencial de inestabilizar nuestro modelo estadístico.

* Además, puede resultar en interpretaciones incorrectas o engañosas sobre la influencia real de cada variable en el resultado principal.

**¿Cómo detectar la multicolinealidad?**

Análisis VIF(Factor de influencia de varianza): Evalúa la medida en que cada variable independiente en un modelo está linealmente relacionada con las otras variables independientes.
Un VIF > 10 indica alta multicolinealidad.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
# quantitative + qual_encoded
vif_data["feature"] = quantitative
vif_data["VIF"] = [variance_inflation_factor(house_df[quantitative].values, i)
                          for i in range(len(quantitative))]
vif_data

In [ ]:
columnas_seleccionadas = vif_data[vif_data.VIF <= 100].feature.tolist()
columnas_seleccionadas

Se deben realizar algunas transformaciones a nuestros datos para reducir este efecto.

# 4.Limpieza de datos.


###  Datos faltantes
Antes de tratar los datos faltantes, es importante determinar su prevalencia y su aleatoriedad, ya que pueden implicar una reducción del tamaño de la muestra.

In [ ]:
# Missing data:
total = house_df.isnull().sum().sort_values(ascending = False)
percent = (house_df.isnull().sum() / house_df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head(20)

In [ ]:
missing = house_df.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
if missing.shape[0] > 0:
  missing.plot.bar()
else:
  print("No tenemos valores nulos.")

### Datos duplicados

In [ ]:
house_df[house_df.duplicated()].shape

In [ ]:
house_df = house_df.drop_duplicates()
house_df.shape

**Conclusión**  
Nuestro dataset no tiene registros duplicados.

### Datos atípicos

In [ ]:
from scipy import stats
import numpy as np

def outliers_col(df, columnas):
  for col in columnas:
    #Solo son outliers los que sobrepasan el z score de 3
    n_outliers = len(df[np.abs(stats.zscore(df[col])) > 3])
    print("{} | {}".format(df[col].name, n_outliers))
outliers_col(house_df, quantitative + ["PrecioVenta"])

Debemos decidir qué hacer con estos Outliers. ¿Serán errores de registros o hechos perfectamente válidos?

**Conclusión**  
- Se han encontrado outliers en todas las variables, con excepción de las variables de fecha.

###Análisis bivariable

También podemos hacer un análisis de outliers de cada variable respecto a la
variable objetivo.

In [ ]:
# Análisis bivariable PrecioVenta/SuperficieHabitable:
var = 'SuperficieHabitable'
house_df.plot.scatter(x = 'SuperficieHabitable', y = 'PrecioVenta', alpha = 0.5);

Tenemos dos valores de superficies habitables que distorsionan nuestros resultados, procedemos a eliminarlos.

In [ ]:
ID_outlier = house_df.sort_values(by = 'SuperficieHabitable', ascending = False)[:2]["Id"].to_list()

In [ ]:
# Eliminación de valores:
house_df = house_df.drop(house_df[house_df['Id'].isin(ID_outlier)].index)

In [ ]:
# Análisis bivariable SalePrice/SuperficieHabitable:
var = 'SuperficieHabitable'
house_df.plot.scatter(x = 'SuperficieHabitable', y = 'PrecioVenta', alpha = 0.5)

**Conclusión**  
- Se han eliminado 2 outliers de la superficie habitable ya que distorsionaban los resultados de nuestro análisis.

# 5.Comprobación de supuestos.

Hay que comprobar cuatro suposiciones fundamentales:

- Normalidad: Los datos deben parecerse a una distribución normal. Es importante porque varias pruebas estadísticas se basan en esta suposición. Si resolvemos la normalidad evitamos otros problemas, como la homocedasticidad.

- Homocedasticidad: Suposición de que las variables dependientes tienen el mismo nivel de varianza en todo el rango de las variables predictoras. La homocedasticidad es deseable porque queremos que el término de error sea el mismo en todos los valores de las variables independientes.

- Linealidad: La forma más común de evaluar la linealidad es examinar los **diagramas de dispersión** y buscar patrones lineales. Si los patrones no son lineales, valdría la pena explorar las transformaciones de datos.

- Ausencia de errores correlacionados - Esto ocurre a menudo en series temporales, donde algunos patrones están relacionados en el tiempo.

### Normalidad

El objetivo es estudiar la variable 'PrecioVenta' de forma fácil, comprobando:

- Histograma: Curtosis y asimetría.
- Gráfica de probabilidad normal: La distribución de los datos debe ajustarse a la diagonal que representa la distribución normal.
-Test de normalidad: Dos pruebas comunes son la Prueba de Shapiro-Wilk y la Prueba de Kolmogorov-Smirnov.


In [ ]:
# Histograma y gráfico de probabilidad normal:
sns.distplot(house_df['PrecioVenta'], fit = norm)
fig = plt.figure()
res = stats.probplot(house_df['PrecioVenta'], plot = plt)

Parece que PrecioVenta no se ajusta a una distribución normal.

De estos gráficos se desprende que 'PrecioVenta' no tiene una distribución normal. Muestra picos, asimetría positiva y no sigue la línea diagonal; aunque una simple transformación de datos puede resolver el problema.

### Test de normalidad  

Los Test de normalidad se utilizan para verificar si una muestra de datos sigue una distribución normal. Dos de los tests más comunes son:
* Test de Shapiro-Wilk:

Es una prueba estadística utilizada para evaluar si una muestra de datos proviene de una distribución normal. Es adecuado para muestras de tamaño moderado($n < 30$). Se plantean las siguientes hipótesis:
  - Hipótesis nula ($H_0$): La muestra sigue una distribución normal.
  - Hipótesis alternativa ($H_1$): La muestra no sigue una distribución normal.
* Test de Kolmogorov-Smirnov:

Es una prueba no paramétrica que determina la **bondad de ajuste** de dos distribuciones de probabilidad entre sí. Es aplicable para muestras grandes.  Para una distribución normal se plantean las siguientes hipótesis:
  - Hipótesis nula ($H_0$): La muestra sigue una distribución normal.
  - Hipótesis alternativa ($H_1$): La muestra no sigue una distribución normal.


¿Los precios de venta se ajustan a una distribución normal?

In [ ]:
# Histograma
sns.distplot(house_df['PrecioVenta'], fit = norm)
plt.plot()

In [ ]:
from scipy import stats
import numpy as np

stat_ks, p_value = sm.stats.diagnostic.kstest_normal(house_df["PrecioVenta"], dist = 'norm')
stat_ks, p_value

p-value = 0.0009999999999998899 < 0.05, por lo tanto los precios de venta de las viviendas no siguen una distribución normal.

In [ ]:
# Transformación de los datos:
house_df['PrecioVenta'] = np.log(house_df['PrecioVenta'])

In [ ]:
# Histograma y gráfico de probabilidad normal sobre los datos transformados:
sns.distplot(house_df['PrecioVenta'], fit = norm)
fig = plt.figure()
res = stats.probplot(house_df['PrecioVenta'], plot = plt)

No se considera que la normalidad sea un requisito indispensable al explorar los datos. En su lugar, se debe buscar y aplicar otros enfoques y herramientas que sean más adecuados para el tipo de datos con los que trabaja y para los objetivos específicos que busca alcanzar. El EDA es un proceso de descubrimiento, no de confirmación.

Nuestra variable 'PrecioVenta' transformada ahora 'parece' seguir una distribución normal.

In [ ]:
import statsmodels.api as sm

stat_ks, p_value = sm.stats.diagnostic.kstest_normal(house_df["PrecioVenta"], dist = 'norm')
stat_ks, p_value

p < 0.05, entonces no se ajusta a una distribución normal.

Conclusión:
Los precios de venta aún no siguen una distribución normal, pero gráficamente se ve que está muy cerca.

## Conclusión general

- Se han revisado las variables clave, analizando el comportamiento de 'PrecioVenta' por sí mismo y junto a las variables más ligadas a él.
- Se ha lidiado con datos faltantes y valores atípicos.
- Se han probado algunos de los supuestos estadísticos fundamentales.
- Se han transformado las variables categóricas.
- Tenemos muchas variables más para analizar.